In [1]:
!python -V

Python 3.10.14


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2024/05/19 22:28:40 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/davidgillespie/projects/mlops_course/week 2/course_work/mlruns/2', creation_time=1716154120108, experiment_id='2', last_update_time=1716154120108, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)



    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('data/green_taxi_data.parquet')
df_val = read_dataframe('data/green_taxi_data_02.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/Users/davidgillespie/opt/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.7587152049492065

In [14]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [15]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

/Users/davidgillespie/opt/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [16]:
import xgboost as xgb

In [17]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [20]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=1,
    trials=Trials()
)

  0%|                                     | 0/1 [00:00<?, ?trial/s, best loss=?]

/Users/davidgillespie/opt/anaconda3/envs/data/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [22:29:10] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.38501                                                    
[1]	validation-rmse:9.12778                                                     
[2]	validation-rmse:8.27425                                                     
[3]	validation-rmse:7.71745                                                     
[4]	validation-rmse:7.35197                                                     
[5]	validation-rmse:7.11119                                                     
[6]	validation-rmse:6.95893                                                     
[7]	validation-rmse:6.85680                                                     
[8]	validation-rmse:6.78283                                                     
[9]	validation-rmse:6.73255                                                     
[10]	validation-rmse:6.70082                                                    
[11]	validation-rmse:6.67779                                                    
[12]	validation-rmse:6.65839

[101]	validation-rmse:6.51217                                                   
[102]	validation-rmse:6.51161                                                   
[103]	validation-rmse:6.51086                                                   
[104]	validation-rmse:6.51022                                                   
[105]	validation-rmse:6.50914                                                   
[106]	validation-rmse:6.50866                                                   
[107]	validation-rmse:6.50796                                                   
[108]	validation-rmse:6.50767                                                   
[109]	validation-rmse:6.50706                                                   
[110]	validation-rmse:6.50628                                                   
[111]	validation-rmse:6.50599                                                   
[112]	validation-rmse:6.50536                                                   
[113]	validation-rmse:6.5046

[202]	validation-rmse:6.46203                                                   
[203]	validation-rmse:6.46218                                                   
[204]	validation-rmse:6.46202                                                   
[205]	validation-rmse:6.46160                                                   
[206]	validation-rmse:6.46129                                                   
[207]	validation-rmse:6.46085                                                   
[208]	validation-rmse:6.46075                                                   
[209]	validation-rmse:6.46044                                                   
[210]	validation-rmse:6.46052                                                   
[211]	validation-rmse:6.46038                                                   
[212]	validation-rmse:6.45994                                                   
[213]	validation-rmse:6.45956                                                   
[214]	validation-rmse:6.4591

[303]	validation-rmse:6.43739                                                   
[304]	validation-rmse:6.43725                                                   
[305]	validation-rmse:6.43685                                                   
[306]	validation-rmse:6.43651                                                   
[307]	validation-rmse:6.43639                                                   
[308]	validation-rmse:6.43643                                                   
[309]	validation-rmse:6.43650                                                   
[310]	validation-rmse:6.43592                                                   
[311]	validation-rmse:6.43577                                                   
[312]	validation-rmse:6.43507                                                   
[313]	validation-rmse:6.43459                                                   
[314]	validation-rmse:6.43470                                                   
[315]	validation-rmse:6.4342

[404]	validation-rmse:6.42221                                                   
[405]	validation-rmse:6.42192                                                   
[406]	validation-rmse:6.42165                                                   
[407]	validation-rmse:6.42166                                                   
[408]	validation-rmse:6.42144                                                   
[409]	validation-rmse:6.42072                                                   
[410]	validation-rmse:6.42028                                                   
[411]	validation-rmse:6.41986                                                   
[412]	validation-rmse:6.41981                                                   
[413]	validation-rmse:6.41997                                                   
[414]	validation-rmse:6.41975                                                   
[415]	validation-rmse:6.41993                                                   
[416]	validation-rmse:6.4199

[505]	validation-rmse:6.41072                                                   
[506]	validation-rmse:6.41109                                                   
[507]	validation-rmse:6.41118                                                   
[508]	validation-rmse:6.41107                                                   
[509]	validation-rmse:6.41110                                                   
[510]	validation-rmse:6.41267                                                   
[511]	validation-rmse:6.41241                                                   
[512]	validation-rmse:6.41216                                                   
[513]	validation-rmse:6.41189                                                   
[514]	validation-rmse:6.41165                                                   
[515]	validation-rmse:6.41175                                                   
[516]	validation-rmse:6.41176                                                   
[517]	validation-rmse:6.4116

/Users/davidgillespie/opt/anaconda3/envs/data/lib/python3.10/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [ ]:
mlflow.xgboost.autolog(disable=True)

In [ ]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        